# Plotting routine for the June2 2022 Event - PSP

In [ ]:
test = 'test'

In [1]:
%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle as p

import datetime as datetime
from datetime import timedelta
import seaborn as sns
import pandas as pd
import os
#import copy as cp

import multiprocess as mp
import py3dcore as py3dcore

import py3dcore.fluxplot as fp

plt.switch_backend('agg')


import warnings
warnings.filterwarnings('ignore')

import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")
#logging.getLogger("sunpy").setLevel("WARNING")
#logging.getLogger("py3dcore_h4c.fluxplot").setLevel("WARNING")

#print(sns.__version__)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


#### setting the times for py3DCORE

In [2]:
t_launch = datetime.datetime(2022, 9, 5, 18, tzinfo=datetime.timezone.utc) # launch time assumed at CME impact at PSP at 14.72 Rs

t_s = datetime.datetime(2022, 9, 7, 8, tzinfo=datetime.timezone.utc) 
t_e = datetime.datetime(2022, 9, 8, 3, tzinfo=datetime.timezone.utc)

t_fit = [
        datetime.datetime(2022, 9, 7, 9, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 7, 15, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 7, 20, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 9, 8, 2, tzinfo=datetime.timezone.utc)
     ]


In [3]:
fittingPoints = []

fittingPoints = [t_fit[i].strftime("%m/%d/%Y, %H:%M:%S") for i in range(len(t_fit))]
print(fittingPoints)

['09/07/2022, 09:00:00', '09/07/2022, 15:00:00', '09/07/2022, 20:00:00', '09/08/2022, 02:00:00']


#### loading the pickle files

In [4]:
path = 'output/solo06092022_512_heeq_1/'

filepath = fp.loadpickle(path, number=-1)

In [5]:
filepath[:-7]

'output/solo06092022_512_heeq_1/07'

#### extracting parameters

In [6]:
res_mineps, res_mean, res_std, ind, keys, allres = fp.get_params(filepath, give_mineps=True)

print(ind)

2023-06-07 13:54:29.069 INFO    py3dcore.fluxplot: Retrieved the following parameters for the run with minimum epsilon:
2023-06-07 13:54:29.070 INFO    py3dcore.fluxplot:  --cme_longitude 125.81
2023-06-07 13:54:29.070 INFO    py3dcore.fluxplot:  --cme_latitude 15.50
2023-06-07 13:54:29.070 INFO    py3dcore.fluxplot:  --cme_inclination 15.21
2023-06-07 13:54:29.071 INFO    py3dcore.fluxplot:  --cme_diameter_1au 0.33
2023-06-07 13:54:29.071 INFO    py3dcore.fluxplot:  --cme_aspect_ratio 3.95
2023-06-07 13:54:29.071 INFO    py3dcore.fluxplot:  --cme_launch_radius 15.32
2023-06-07 13:54:29.072 INFO    py3dcore.fluxplot:  --cme_launch_velocity 1047.31
2023-06-07 13:54:29.073 INFO    py3dcore.fluxplot:  --t_factor 198.76
2023-06-07 13:54:29.073 INFO    py3dcore.fluxplot:  --cme_expansion_rate 1.14
2023-06-07 13:54:29.073 INFO    py3dcore.fluxplot:  --magnetic_decay_rate 1.64
2023-06-07 13:54:29.073 INFO    py3dcore.fluxplot:  --magnetic_field_strength_1au 6.13
2023-06-07 13:54:29.074 INFO  

Results :

	cme_longitude: 		122.40 +/- 5.74
	cme_latitude: 		-1.52 +/- 8.33
	cme_inclination: 		20.55 +/- 47.98
	cme_diameter_1au: 		0.30 +/- 0.02
	cme_aspect_ratio: 		3.02 +/- 0.66
	cme_launch_radius: 		14.97 +/- 0.53
	cme_launch_velocity: 		1405.48 +/- 328.54
	t_factor: 		161.74 +/- 45.97
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		6.44 +/- 0.69
	background_drag: 		1.62 +/- 0.72
	background_velocity: 		613.52 +/- 56.56
377


In [7]:
out = fp.get_ensemble_stats(filepath)
out.describe()

,lon,lat,inc,D1AU,delta,launch radius,launch speed,t factor,expansion rate,B decay rate,B1AU,gamma,vsw
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.00000,512.000000,512.000000,512.000000
mean,122.404404,-1.519575,20.545403,0.300723,3.021212,14.965982,1405.479980,161.734940,1.140000,1.64000,6.436749,1.619575,613.524170
std,5.749835,8.342556,48.025894,0.021043,0.663556,0.529226,328.863892,46.018162,0.000008,0.00001,0.685963,0.725023,56.613281
min,106.349731,-30.552116,0.032050,0.236411,1.009933,14.016193,808.868835,24.668550,1.140000,1.64000,5.004392,0.200123,351.334473
25%,118.640852,-7.050511,9.476101,0.284935,2.565984,14.503666,1134.897888,130.753860,1.140000,1.64000,5.950719,1.003611,579.466888
50%,122.062801,-1.473212,13.795054,0.298673,3.128043,14.989139,1385.524536,162.439217,1.140000,1.64000,6.411795,1.642444,619.552917
75%,125.653366,4.139654,18.349843,0.316026,3.545613,15.387470,1680.444550,198.775810,1.140000,1.64000,6.880287,2.212752,657.744781
max,141.129547,20.634428,359.631927,0.349388,3.990799,15.994086,1989.345215,249.447784,1.140000,1.64000,8.692548,2.997389,699.545593


In [ ]:
# write output of describe() to csv-file
out.describe().to_csv(filepath+'parameters.csv')

# write parameters of whole ensemble to txt-file
with open(filepath+'parameters_ensemble.txt', 'a') as f:
    dfAsString = out.to_string(header=True, index=True)
    f.write(dfAsString)

In [ ]:
# write output of describe() to txt-file    
with open(filepath+'parameters_summary.txt', 'a') as f:
    f.write('Summary of results')
    f.write('\n\n')
    f.write('Results:')
    f.write('\n')
    for i in range(1, len(keys)):    
        f.write("\t{}: \t\t{:.02f} +/- {:.02f}".format(keys[i], res_mean[i], res_std[i]))
        f.write('\n')
    f.write('\n')
    f.write('Parameters for run with min(eps):')
    f.write('\n')
    for i in range(1, len(keys)):
        f.write(" --{} {:.2f}".format(keys[i], res_mineps[i]))
        f.write('\n')
    f.write('\n')
    f.write('Fitting points:')
    f.write(str(fittingPoints))
    f.write('\n')

#### loading and plotting the fitting results

In [ ]:
t_s = datetime.datetime(2022, 9, 7, 7, tzinfo=datetime.timezone.utc)
t_e = datetime.datetime(2022, 9, 8, 4, tzinfo=datetime.timezone.utc)

In [ ]:
fp.fullinsitu(observer='SOLO', t_fit=t_fit, start=t_s, end=t_e, filepath=filepath, ref_frame='HEEQ', best=False, 
              ensemble=True, save_fig=True, legend=True, max_index=512, title=False, fit_points=True)

In [ ]:
fp.scatterparams(filepath)

In [ ]:
print(out['lon'].mean())
print(out['lon'].std())
print(out['launch radius'].std())
print(out['launch speed'].std())

In [ ]:
print(res_mineps[1])

#### Plotting the fluxrope and spacecraft positions

In [8]:
#measurement times 
#tm0 =  datetime.datetime(2022, 6, 2, 6)
tm1 =  t_launch + datetime.timedelta(hours=40)
#tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d(spacecraftlist=['solo'], planetlist=['Earth'], t=tm1, filepath=filepath, legend=False, view_azim=res_mineps[1]-90, title=False,
                view_elev=0, view_radius=0.7, index=0)

Results :

	cme_longitude: 		122.40 +/- 5.74
	cme_latitude: 		-1.52 +/- 8.33
	cme_inclination: 		20.55 +/- 47.98
	cme_diameter_1au: 		0.30 +/- 0.02
	cme_aspect_ratio: 		3.02 +/- 0.66
	cme_launch_radius: 		14.97 +/- 0.53
	cme_launch_velocity: 		1405.48 +/- 328.54
	t_factor: 		161.74 +/- 45.97
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		6.44 +/- 0.69
	background_drag: 		1.62 +/- 0.72
	background_velocity: 		613.52 +/- 56.56


2023-06-07 13:54:37.178 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


observer: Solar Orbiter
t_snap: 2022-09-07 10:00:00+00:00


### make fluxrope 3d movie

In [11]:
def make_frame(k):
    tm0 = t_launch
    t = tm0 + k*datetime.timedelta(hours=1)
    
    fig = fp.full3d(spacecraftlist=['psp', 'solo'], planetlist=['Earth'], legend=False, title=False,
              t=t, filepath=filepath, save_fig=True, view_radius=0.7, view_elev=90, view_azim=res_mineps[1]-90)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d/' + frmstr + '.jpg', dpi=300)  
    return fig
    

In [12]:
inn = [i for i in range(100)]


mpool = mp.Pool(processes=2)
mpool.map(make_frame, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = mp.Pool(processes=2)
mpool.map(make_frame, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = mp.Pool(processes=2)
mpool.map(make_frame, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = mp.Pool(processes=2)
mpool.map(make_frame, inn[60:100])
mpool.close()
mpool.join()

print('done 4/4')

Results :

Results :
	cme_longitude: 		122.40 +/- 5.74

	cme_latitude: 		-1.52 +/- 8.33	cme_longitude: 		122.40 +/- 5.74

	cme_inclination: 		20.55 +/- 47.98	cme_latitude: 		-1.52 +/- 8.33

	cme_diameter_1au: 		0.30 +/- 0.02	cme_inclination: 		20.55 +/- 47.98

	cme_aspect_ratio: 		3.02 +/- 0.66	cme_diameter_1au: 		0.30 +/- 0.02

	cme_launch_radius: 		14.97 +/- 0.53
	cme_aspect_ratio: 		3.02 +/- 0.66
	cme_launch_velocity: 		1405.48 +/- 328.54	cme_launch_radius: 		14.97 +/- 0.53
	t_factor: 		161.74 +/- 45.97
	cme_launch_velocity: 		1405.48 +/- 328.54

	cme_expansion_rate: 		1.14 +/- 0.00	t_factor: 		161.74 +/- 45.97

	cme_expansion_rate: 		1.14 +/- 0.00	magnetic_decay_rate: 		1.64 +/- 0.00

	magnetic_decay_rate: 		1.64 +/- 0.00	magnetic_field_strength_1au: 		6.44 +/- 0.69
	magnetic_field_strength_1au: 		6.44 +/- 0.69
	background_drag: 		1.62 +/- 0.72

	background_drag: 		1.62 +/- 0.72	background_velocity: 		613.52 +/- 56.56

	background_velocity: 		613.52 +/- 56.56
observer: Solar Orbite

2023-06-07 13:59:34.997 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


observer:t_snap: 2022-09-05 21:00:00+00:00
 Solar Orbiter


2023-06-07 13:59:35.029 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


observer: Parker Solar Probet_snap: 2022-09-05 18:00:00+00:00

observer: Parker Solar Probe


...

t_snap: 2022-09-09 21:00:00+00:00
observer: Solar Orbiter


2023-06-07 14:00:30.550 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


t_snap: 2022-09-09 16:00:00+00:00
observer: Parker Solar Probe


2023-06-07 14:00:30.562 INFO    py3dcore.fluxplot: Using HelioSat to retrieve observer data


t_snap: 2022-09-09 16:00:00+00:00
done 4/4


In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_movie.mp4 -y -loglevel quiet')

### plot fluxrope 3d multiview

In [ ]:
fig = fp.full3d_multiview(tm1, filepath=filepath)

### make fluxrope 3d multiview movie

In [ ]:
def make_frame2(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    frametime = k
    
    fig = fp.full3d_multiview_movie(t_launch = tm0, t = t, filepath=filepath,
                                    frametime=k)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d_multiview/'+frmstr+'.jpg',dpi=200)  
    return fig

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d_multiview/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_multiview_movie.mp4 -y -loglevel quiet')

# Used for Debug

In [ ]:
TP_A =  t_launch + datetime.timedelta(hours=2)
TP_B =  t_launch + datetime.timedelta(hours=40)


C_A = "xkcd:red"
C_B = "xkcd:blue"
    
sns.set_style('whitegrid')

fig = plt.figure(figsize=(15, 11),dpi=100)

ax1 = plt.subplot2grid((2, 3), (0, 0),rowspan=2,colspan=2,projection='3d')  
ax2 = plt.subplot2grid((2, 3), (0, 2),projection='3d')  
ax3 = plt.subplot2grid((2, 3), (1, 2),projection='3d')  

model_obj = fp.returnmodel(filepath)


fp.plot_configure(ax1, view_azim=150, view_elev=25, view_radius=.2,light_source=True) #view_radius=.08

fp.plot_3dcore(ax1, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax1, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax1,0.31,-0.25,0.0,-0.2)


fp.plot_configure(ax2, view_azim=165-90, view_elev=90, view_radius=.08,light_source=True)
    
fp.plot_3dcore(ax2, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax2, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax2,0.26,-0.41,0.08,0.0)



fp.plot_configure(ax3, view_azim=65, view_elev=-5, view_radius=.01,light_source=True)
    
fp.plot_3dcore(ax3, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax3, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax3,0.26,-0.41,0.08,0.0)

